In [ ]:
# 1. Import your modules
from ecosystem_model import EcosystemModel
import ecosystem_plotting as plot

# 2. Setup
model = EcosystemModel(height=50, width=50)
model.initialize_grid()

# 3. Run Simulation
history = []
print("Starting Simulation...")

for i in range(200):
    grid = model.step()

    if i % 10 == 0:
        state = model.get_state()
        # Calculate mean biomass for history
        mean_A = state[:, :, 14].mean()
        mean_B = state[:, :, 15].mean()
        history.append((mean_A, mean_B))

    # Optional: Plot every 50 steps to see progress
    if i % 50 == 0:
        plot.plot_state(state, timepoint=f"(Step {i})")

# 4. Final Analysis
print("Simulation Complete.")
plot.plot_state(model.get_state(), timepoint="(Final)")
plot.plot_history(history)